# Loading Package

In [ ]:
import os

# Limit the number of threads for underlying BLAS/OpenMP libraries
os.environ["OMP_NUM_THREADS"] = "64"         # OpenMP threads
os.environ["OPENBLAS_NUM_THREADS"] = "64"      # OpenBLAS threads
os.environ["MKL_NUM_THREADS"] = "64"           # MKL threads
os.environ["NUMEXPR_NUM_THREADS"] = "64"       # NumExpr threads

In [ ]:
import SPIX
from SPIX.image_processing.image_cache import *
import pandas as pd
import scanpy as sc
import squidpy as sq
import numpy as np
import squidpy as sq
import matplotlib.pyplot as plt
import anndata as ad


# Multiscale Workflow - 2um VisiumHD CRC data

In [ ]:
SPIX

In [ ]:
adata = ad.read_zarr('/home/Data_Drive_8TB/chs1151/clean_up/SPIX/CRC_P2_2um')

In [ ]:
adata = SPIX.tm.generate_embeddings(
    adata,
    dim_reduction='PCA',
    normalization='log_norm', 
    n_jobs=32,
    dimensions=30,
    nfeatures=2000,
    use_coords_as_tiles=True,
    coords_max_gap_factor=None,
    force=True,
    use_hvg_only=True,
    raster_stride=10,
    filter_threshold=1,
    raster_max_pixels_per_tile=400,
    raster_random_seed=42
)


In [ ]:
adata = SPIX.ip.smooth_image(
    adata,
    methods=['graph','gaussian'],
    embedding='X_embedding',
    embedding_dims = list(range(30)),
    graph_k=30,
    graph_t=20,
    gaussian_sigma=300,
    n_jobs=10,
    rescale_mode='final',
)

In [ ]:
adata = SPIX.ip.equalize_image(
    adata,
    dimensions=list(range(30)),
    embedding='X_embedding_smooth',
    sleft=5,sright=5
)

In [ ]:
cache_embedding_image(adata, embedding='X_embedding_equalize',
                            dimensions=list(range(30)), 
                            key='image_plot_slic',
                            origin=True,
                            figsize=(30,30),
                            fig_dpi=100,
                            verbose=False,show=True)

In [ ]:
SPIX.sp.segment_image(
    adata,
    dimensions=list(range(30)),
     embedding='X_embedding_equalize',
     method='image_plot_slic',
    pitch_um=2,
    target_segment_um=500,
     compactness=0.5,
     verbose=True,
    figsize = (30, 30),
    use_cached_image=True,
    enforce_connectivity=False,
     origin=True,
    show_image=True
)

In [ ]:
SPIX.pl.image_plot(
    adata,
    dimensions=[0, 1, 2],
    embedding='X_embedding_segment',
    boundary_method='pixel',
    figsize=(10, 10),     # Adjust figure size here
    fixed_boundary_color='black',
    boundary_linewidth=1,
    alpha=1,
    plot_boundaries=True,
    origin=True
)

In [ ]:
sq.gr.spatial_neighbors(adata, coord_type='generic')

In [ ]:
# multiscale_svg_advanced2.py
import os, gc, tempfile, shutil, warnings, itertools
import numpy as np, pandas as pd, scanpy as sc, seaborn as sns, matplotlib.pyplot as plt
from joblib import Parallel, delayed
from scipy.stats import rankdata
from sklearn.mixture import GaussianMixture

#################################
# 1) Setting
#################################
resolutions    = [2,8,16,30,50,100,250,500]
compactnesses  = [0.5]
dims_use       = list(range(30))
segment_method = 'image_plot_slic'
embedding_key  = 'X_embedding_equalize'
use_cached_image=True,
moran_thresh   = 0      # MoranI threshold
n_jobs         = 3
top_changed    = 100
use_memmap     = False    # True: h5ad 

#################################
# 2) single-scale calculation function
#################################
def calc_scale(adata,scale_id, res, comp, adata_path=None):
    if adata_path:
        ad = sc.read_h5ad(adata_path)
    else:
        ad = adata.copy()
    # segmentation
    SPIX.sp.segment_image(
        ad,
        dimensions=dims_use,
        embedding=embedding_key,
        method=segment_method,
        target_segment_um=res,
        compactness=comp,
        figsize = (30, 30),
        enforce_connectivity=False,
        use_cached_image=use_cached_image,
        origin=True,
        verbose=False
    )
    # pseudo-bulk Moran
    _, moran = SPIX.an.perform_pseudo_bulk_analysis(
        ad,
        segment_key='Segment',
        normalize_total=True,
        log_transform=True,
        expr_agg='sum',
        moranI_threshold=moran_thresh,
        segment_graph_strategy='collapsed',
        collapse_row_normalize=True,
        perform_pca=False,
        highly_variable=False,
        mode='moran'
    )
    if moran.empty:
        warnings.warn(f"[{scale_id}] No MoranI result ")
        return pd.DataFrame()
    # rank 
    moran['rank_'+scale_id] = rankdata(-moran['I'], method='min')
    return moran[['rank_'+scale_id]]

#################################
# 3) parallel calculation
#################################
param_grid = list(itertools.product(resolutions, compactnesses))
print(f"▶ Total {len(param_grid)} scales ")

adata_path = None
if use_memmap:
    td = tempfile.mkdtemp()
    adata_path = os.path.join(td, 'adata_tmp.h5ad')
    adata.write_h5ad(adata_path)

results = Parallel(n_jobs=n_jobs, backend='loky', verbose=10)(
    delayed(calc_scale)(adata,f"r{r}_c{c}", r, c, adata_path)
    for r, c in param_grid
)

if use_memmap:
    shutil.rmtree(td)



In [ ]:

# concat rank table
rank_tables = [df for df in results if not df.empty]

rank_mat = pd.concat(rank_tables, axis=1, join='outer')
max_rank = int(rank_mat.max().max())
rank_mat = rank_mat.fillna(max_rank + 1)

rank_mat.sort_index(axis=1, inplace=True)  

#################################
# 5) resolution / compactness axis
#################################
# resolution axis
res_rank = pd.DataFrame(index=rank_mat.index)
for r in resolutions:
    cols = [f"rank_r{r}_c{c}" for c in compactnesses]
    res_rank[f"res_{r}"] = rank_mat[cols].mean(axis=1)

# compactness axis
comp_rank = pd.DataFrame(index=rank_mat.index)
for c in compactnesses:
    cols = [f"rank_r{r}_c{c}" for r in resolutions]
    comp_rank[f"comp_{c}"] = rank_mat[cols].mean(axis=1)


In [ ]:
df = res_rank.copy()

In [ ]:
import pandas as pd

# Define regions by column names
early_cols = ['res_2', 'res_8','res_16']
mid_cols   = ['res_30','res_50','res_100']
late_cols  = ['res_250','res_500']


# Compute mean rank in each region
df['mean_early'] = df[early_cols].mean(axis=1)
df['mean_mid']   = df[mid_cols].mean(axis=1)
df['mean_late']  = df[late_cols].mean(axis=1)

# Set threshold ratio: difference between 1st and 2nd smallest mean must be >= this
# threshold_ratio = 0.7  # e.g., 20%
threshold_ratio = 0.93  # e.g., 20%

def categorize_with_threshold(row):
    # collect region means
    regions = {
        'early': row['mean_early'],
        'mid':   row['mean_mid'],
        'late':  row['mean_late']
    }
    # sort regions by mean ascending
    sorted_regs = sorted(regions.items(), key=lambda x: x[1])
    # lowest and second-lowest
    (reg1, val1), (reg2, val2) = sorted_regs[0], sorted_regs[1]
    # check relative difference
    # if absolute or relative difference is big enough, assign; else mixed
    if (val2 - val1) / val2 >= threshold_ratio:
        return reg1
    else:
        return 'mixed'

# Apply categorization
df['category'] = df.apply(categorize_with_threshold, axis=1)

# Optional: view how many genes fall in each category
print(df['category'].value_counts())

# Optional: inspect sample
print(df[['mean_early','mean_mid','mean_late','category']].head())


In [ ]:
def plot_traj(df, genes, xlabel, title):
    plt.figure(figsize=(8,4))
    x = np.arange(df.shape[1])
    for g in genes:
        plt.plot(x, df.loc[g], '-o', alpha=0.7, label=g)
    plt.gca().invert_yaxis()
    plt.xticks(x, df.columns, rotation=45)
    plt.xlabel(xlabel); plt.ylabel('Rank')
    plt.title(title)
    plt.legend(bbox_to_anchor=(1.02,1), loc='upper left', fontsize=8)
    plt.tight_layout(); plt.show()

In [ ]:
df['late_m_mid'] = df['mean_late']-df['mean_mid']

In [ ]:
plot_traj(res_rank,df[df['category']=='early'].sort_values('mean_early').head(10).index.tolist(), 'Resolution', 'Unique in high resolution')

In [ ]:
plot_traj(res_rank,df[df['category']=='late'].sort_values('mean_late').head(10).index.tolist(), 'Resolution', 'Unique in low resolution')

In [ ]:
plot_traj(res_rank,df[df['category']=='late'].sort_values('mean_mid',ascending=False).head(10).index.tolist(), 'Resolution', 'Unique in low resolution')

In [ ]:
plot_traj(res_rank,df[df['category']=='early'].sort_values('mean_mid',ascending=False).head(10).index.tolist(), 'Resolution', 'Unique in middle resolution')

In [ ]:
gene = 'LYVE1'

In [ ]:
SPIX.an.add_gene_expression_embedding(adata,genes=[gene],segment_key=None,normalize_total=True,log1p=True)

In [ ]:
SPIX.pl.image_plot(
    adata,
    dimensions=[0],
    embedding='X_gene_embedding',
    boundary_method='pixel',
    # imshow_scale_factor=1,
    figsize=(10, 10),     # Adjust figure size here
    # scaling_factor=2,    # Adjust point size here
    fixed_boundary_color='Black',
    cmap='viridis',
    boundary_linewidth=1,
    show_colorbar=True,
    prioritize_high_values=True,
    title=gene,
    alpha=1,
    plot_boundaries=False,
    origin=True
)

In [ ]:
SPIX.an.add_gene_expression_embedding(adata,genes=[gene],segment_key='Segment',normalize_total=True,log1p=True)

In [ ]:
SPIX.pl.image_plot(
    adata,
    dimensions=[0],
    embedding='X_gene_embedding',
    boundary_method='pixel',
    figsize=(10, 10),     # Adjust figure size here
    # scaling_factor=2,    # Adjust point size here
    fixed_boundary_color='Black',
    cmap='viridis',
    boundary_linewidth=1,
    show_colorbar=True,
    prioritize_high_values=True,
    title=gene,
    alpha=1,
    plot_boundaries=False,
    origin=True
)

In [ ]:
# ==============================================================
# Parallel SPIX grids using in-memory `adata` (no disk reload)
# - Works best on Linux with 'fork' start method.
# - Each worker process inherits the parent's memory (copy-on-write),
#   so we can use the global `adata` directly without reloading.
# ==============================================================

import os
import math
import warnings
import gc
from typing import List, Tuple
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import pandas as pd
from concurrent.futures import ProcessPoolExecutor, as_completed
import multiprocessing as mp

# Try to enforce 'fork' (Linux). On macOS/Windows this may fail and fall back.
try:
    mp.set_start_method("fork", force=True)
except RuntimeError:
    # Already set or platform doesn't support 'fork'
    pass

# -----------------------------
# 0) Helpers: ranking + lists
# -----------------------------


# -----------------------------------------
# 1) Worker: render one gene (uses global adata)
# -----------------------------------------
def _render_gene_png_worker(gene: str,
                            out_png: str,
                            segment_key: str,
                            normalize_total: bool,
                            log1p: bool,
                            title_prefix: str,
                            tile_figsize: tuple,
                            tile_dpi: int):
    """
    Executed in a child process.
    Uses globally inherited `adata` and `SPIX` (via fork on Linux).
    - Compute single-gene embedding -> dimensions=[0]
    - Plot and save PNG
    """
    # Import here so that child process resolves the same modules
    import SPIX
    import matplotlib.pyplot as plt

    global adata  # inherited via fork (Linux). On spawn, this may copy/pickle (slower).

    # Gracefully handle missing gene
    if str(gene) not in set(map(str, adata.var_names)):
        fig = plt.figure(figsize=tile_figsize)
        plt.text(0.5, 0.5, f"{title_prefix}{gene}\n(not in var_names)", ha='center', va='center')
        plt.axis('off')
        fig.savefig(out_png, dpi=tile_dpi, bbox_inches='tight')
        plt.close(fig)
        return out_png

    # Compute single-gene embedding (overwrites X_gene_embedding in THIS process only)
    SPIX.an.add_gene_expression_embedding(
        adata,
        genes=[gene],
        segment_key=segment_key,
        normalize_total=normalize_total,
        log1p=log1p
    )

    # Plot using the single dimension [0]
    SPIX.pl.image_plot(
        adata,
        dimensions=[0],                  # single dim only
        embedding='X_gene_embedding',
        boundary_method='pixel',
        imshow_tile_size=10,
        imshow_scale_factor=1,
        figsize=tile_figsize,
        fixed_boundary_color='Black',
        cmap='viridis',
        boundary_linewidth=1,
        show_colorbar=True,
        prioritize_high_values=True,
        title=f"{title_prefix}{gene}",
        alpha=1,
        plot_boundaries=False,
        origin=True
    )
    plt.savefig(out_png, dpi=tile_dpi, bbox_inches='tight')
    plt.close()

    # Cleanup in child
    gc.collect()
    return out_png

# ------------------------------
# 2) Grid assembly
# ------------------------------
def _assemble_grid(tile_paths: List[str], out_path: str, cols: int = 5, suptitle: str = "", dpi: int = 150):
    """Assemble saved PNG tiles into a grid figure."""
    rows = math.ceil(len(tile_paths)/cols) if tile_paths else 1
    fig, axes = plt.subplots(rows, cols, figsize=(cols*4, rows*4), squeeze=False)
    if suptitle:
        fig.suptitle(suptitle, fontsize=16)
    i = 0
    for r in range(rows):
        for c in range(cols):
            ax = axes[r][c]
            ax.axis('off')
            if i < len(tile_paths):
                img = Image.open(tile_paths[i])
                ax.imshow(img)
            i += 1
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.savefig(out_path, dpi=dpi, bbox_inches='tight')
    plt.close(fig)

# -------------------------------------------------------
# 3) Orchestrator: real parallel pipeline per group
# -------------------------------------------------------
def build_grid_for_group_parallel(group_name: str,
                                  genes: List[str],
                                  out_dir: str = "./spix_grids",
                                  cols: int = 5,
                                  tile_figsize=(10,10),
                                  tile_dpi=150,
                                  segment_key='Segment',
                                  normalize_total=True,
                                  log1p=True,
                                  max_workers=6):
    """Run per-gene SPIX rendering in parallel (per-process), then assemble grid."""
    if not genes:
        warnings.warn(f"[{group_name}] No genes to render.")
        return None

    os.makedirs(out_dir, exist_ok=True)
    tmp_dir = os.path.join(out_dir, f"tiles_{group_name}")
    os.makedirs(tmp_dir, exist_ok=True)

    out_png = os.path.join(out_dir, f"grid_{group_name}.png")

    # Submit jobs
    futures = []
    ordered_tiles = [os.path.join(tmp_dir, f"{g}.png") for g in genes]
    with ProcessPoolExecutor(max_workers=max_workers) as ex:
        for g, png in zip(genes, ordered_tiles):
            futures.append(
                ex.submit(
                    _render_gene_png_worker,
                    g, png,
                    segment_key, normalize_total, log1p,
                    f"{group_name} | ",
                    tile_figsize, tile_dpi
                )
            )
        # Ensure all complete (also catches exceptions early)
        for f in as_completed(futures):
            _ = f.result()

    # Assemble in original order
    existing_tiles = [p for p in ordered_tiles if os.path.exists(p)]
    _assemble_grid(existing_tiles, out_path=out_png, cols=cols, suptitle=f"{group_name} (n={len(existing_tiles)})", dpi=tile_dpi)

    print(f"[Saved] {out_png}")
    return out_png


In [ ]:
matplotlib.use("Agg")  # no GUI backend inside workers
# ==============================================================
# 4) RUN: make top lists & build grids
# ==============================================================

TOP_K = 10                 # number of genes per group
GROUP_COLS = 5             # columns in grid
TILE_FIGSIZE = (10, 10)    # per-tile SPIX figure size
TILE_DPI = 400
MAX_WORKERS = 10
SEGMENT_KEY = "Segment"
NORMALIZE_TOTAL = True
LOG1P = True

OUT_DIR = "./spix_1007_MOSTA_multiscale"

build_grid_for_group_parallel(
    group_name=f"moran_late_500_top{TOP_K}",
    genes=df[df['category']=='late'].sort_values('mean_late').head(10).index.tolist(),
    out_dir=OUT_DIR,
    cols=GROUP_COLS,
    tile_figsize=TILE_FIGSIZE,
    tile_dpi=TILE_DPI,
    segment_key=SEGMENT_KEY,
    normalize_total=NORMALIZE_TOTAL,
    log1p=LOG1P,
    max_workers=MAX_WORKERS
)

build_grid_for_group_parallel(
    group_name=f"moran_late_m_mid_500_top{TOP_K}",
    genes=df[df['category']=='late'].sort_values('late_m_mid',ascending=True).head(10).index.tolist(),
    out_dir=OUT_DIR,
    cols=GROUP_COLS,
    tile_figsize=TILE_FIGSIZE,
    tile_dpi=TILE_DPI,
    segment_key=SEGMENT_KEY,
    normalize_total=NORMALIZE_TOTAL,
    log1p=LOG1P,
    max_workers=MAX_WORKERS
)

import matplotlib
import matplotlib.pyplot as plt

# Switch back to Jupyter's inline backend
plt.switch_backend('module://matplotlib_inline.backend_inline')


# Celltype transparency - 8um VisiumHD CRC data

In [ ]:
adata_8um = sc.read_h5ad('/home/NAS/chs1151/C_D1/square_008um/CRC_HD_P2_8um.adata')

In [ ]:
adata_8um.obsm['spatial'] = adata_8um.obsm['spatial'].astype('float')

In [ ]:
adata_8um = SPIX.tm.generate_embeddings(
    adata_8um,
    dim_reduction='PCA',
    normalization='log_norm', 
    n_jobs=32,
    dimensions=30,
    nfeatures=2000,
    use_coords_as_tiles=True,
    coords_max_gap_factor=None,
    force=True,
    use_hvg_only=True,
    raster_stride=10,
    filter_threshold=1,
    raster_max_pixels_per_tile=400,
    raster_random_seed=42
)


In [ ]:
adata_8um = SPIX.ip.smooth_image(
    adata_8um,
    methods=['graph','gaussian'],
    embedding='X_embedding',
    embedding_dims = list(range(30)),
    graph_k=30,
    graph_t=3,
    gaussian_sigma=50,
    n_jobs=10,
    rescale_mode='final',
)

In [ ]:
adata_8um = SPIX.ip.equalize_image(
    adata_8um,
    dimensions=list(range(30)),
    embedding='X_embedding_smooth',
    sleft=5,sright=5
)

In [ ]:
SPIX.pl.image_plot(
    adata_8um,
    dimensions=[0, 1, 2],
    embedding='X_embedding_equalize',
    figsize=(10, 10),     
    plot_boundaries=False,
    origin=True
)

In [ ]:
SPIX.sp.segment_image(
    adata_8um,
    dimensions=list(range(30)),
     embedding='X_embedding_equalize',
     method='image_plot_slic',
    pitch_um=8,
    target_segment_um=500,
     compactness=1,
     verbose=True,
    figsize = (10, 10),
    enforce_connectivity=False,
     origin=True,
    show_image=True
)

In [ ]:
import celltypist
from celltypist import models

In [ ]:
adata_8um.layers['counts'] = adata_8um.X.copy()

In [ ]:
sc.pp.normalize_total(adata_8um,target_sum=1e4)
sc.pp.log1p(adata_8um)

In [ ]:
predictions_8bin_crc = celltypist.annotate(adata_8um, model = 'Human_Colorectal_Cancer.pkl', majority_voting = False,use_GPU=True)
adata_8um.obs['predicted_labels'] = predictions_8bin_crc.predicted_labels['predicted_labels'].copy()

In [ ]:
adata_8um.X = adata_8um.layers['counts'].copy()

In [ ]:
import pandas as pd

# Bing group mapping
big_group_map = {
    # Immune
    "CD4+ T cells": "Immune",
    "CD8+ T cells": "Immune",
    "Regulatory T cells": "Immune",
    "T follicular helper cells": "Immune",
    "T helper 17 cells": "Immune",
    "gamma delta T cells": "Immune",
    "CD19+CD20+ B": "Immune",
    "IgA+ Plasma": "Immune",
    "IgG+ Plasma": "Immune",
    "NK cells": "Immune",
    "cDC": "Immune",
    "Mast cells": "Immune",

    # Epithelial
    "Mature Enterocytes type 1": "Epithelial",
    "Mature Enterocytes type 2": "Epithelial",
    "Goblet cells": "Epithelial",
    "Enteric glial cells": "Epithelial",
    "Stem-like/TA": "Epithelial",
    "Intermediate": "Epithelial",
    "Proliferative ECs": "Epithelial",
    "Stalk-like ECs": "Epithelial",
    "Tip-like ECs": "Epithelial",
    # CMS groups
    "CMS1": "CMS subtype",
    "CMS2": "CMS subtype",
    "CMS3": "CMS subtype",
    "CMS4": "CMS subtype",
    # "CMS1": "Epithelial",
    # "CMS2": "Epithelial",
    # "CMS3": "Epithelial",
    # "CMS4": "Epithelial",

    # Stromal
    "Myofibroblasts": "Stromal",
    "Smooth muscle cells": "Stromal",
    "Pericytes": "Stromal",
    "Stromal 1": "Stromal",
    "Stromal 2": "Stromal",
    "Stromal 3": "Stromal",

    # Endothelial / vascular
    "Lymphatic ECs": "Endothelial/Vascular",
    "Proliferating": "Endothelial/Vascular",

    # Special / functional
    "SPP1+": "SPP1+",
    "Pro-inflammatory": "Pro-inflammatory",
    
    "Unknown": "Unknown"
}

# Small group mapping
small_group_map = {
    # T cells
    "CD4+ T cells": "T cell subset",
    "CD8+ T cells": "T cell subset",
    "Regulatory T cells": "T cell subset",
    "T follicular helper cells": "T cell subset",
    "T helper 17 cells": "T cell subset",
    "gamma delta T cells": "T cell subset",

    # B lineage
    "CD19+CD20+ B": "B cell",
    "IgA+ Plasma": "Plasma cell",
    "IgG+ Plasma": "Plasma cell",

    # Innate immune
    "NK cells": "Innate immune",
    "cDC": "Innate immune",
    "Mast cells": "Innate immune",

    # Epithelial – differentiated
    "Mature Enterocytes type 1": "Mature epithelial",
    "Mature Enterocytes type 2": "Mature epithelial",
    "Goblet cells": "Mature epithelial",
    "Enteric glial cells": "Mature epithelial",

    # Epithelial – progenitor
    "Stem-like/TA": "Progenitor epithelial",
    "Intermediate": "Progenitor epithelial",

    # Epithelial – proliferative
    "Proliferative ECs": "Proliferative epithelial",
    "Stalk-like ECs": "Proliferative epithelial",
    "Tip-like ECs": "Proliferative epithelial",

    # CMS groups
    "CMS1": "CMS subtype",
    "CMS2": "CMS subtype",
    "CMS3": "CMS subtype",
    "CMS4": "CMS subtype",

    # Stromal
    "Myofibroblasts": "Fibroblast-like",
    "Smooth muscle cells": "Smooth muscle",
    "Pericytes": "Perivascular cell",
    "Stromal 1": "Stromal subtype",
    "Stromal 2": "Stromal subtype",
    "Stromal 3": "Stromal subtype",

    # Endothelial / vascular
    "Lymphatic ECs": "Endothelial cell",
    "Proliferating": "Proliferating cell",
    # Special / functional
    "SPP1+": "SPP1+",
    "Pro-inflammatory": "Pro-inflammatory",
    "Unknown": "Unknown"
}



In [ ]:
adata_8um.obs["big_group"] = adata_8um.obs["predicted_labels"].map(big_group_map)
adata_8um.obs["small_group"] = adata_8um.obs["predicted_labels"].map(small_group_map)


In [ ]:
SPIX.pl.image_plot(
    adata_8um,
    embedding='X_embedding',
    dimensions=[0, 1, 2],
    color_by='small_group',
    palette='sc.pl.palettes.default_20',
    figsize=(20, 20),     
    show_legend=True,
    legend_ncol=1,
    plot_boundaries=False,
    origin=True
)

In [ ]:
gene = 'SLC4A4'

In [ ]:
SPIX.an.add_gene_expression_embedding(adata_8um,genes=[gene],segment_key=None,normalize_total=True,log1p=True)

In [ ]:
SPIX.pl.image_plot(
    adata_8um,
    dimensions=[0],
    embedding='X_gene_embedding',
    boundary_method='pixel',
    # imshow_tile_size=10,
    figsize=(10, 10),     
    fixed_boundary_color='Black',
    cmap='viridis',
    boundary_linewidth=1,
    show_colorbar=True,
    prioritize_high_values=True,
    title=gene,
    alpha=1,
    plot_boundaries=False,
    origin=True
)

In [ ]:
SPIX.pl.image_plot(
    adata_8um,
    dimensions=[0],
    embedding='X_gene_embedding',
    color_by='small_group',
    boundary_method='pixel',
    palette='sc.pl.palettes.default_20',
    show_legend=True,
    legend_ncol=1,
    overlap_priority=None,
    imshow_scale_factor=2,
    title=gene,
    cmap='viridis',
    figsize=(20, 20),     
    segment_color_by_major=False,
    segment_show_pie=False,
    segment_annotate_by='big_group', 
    segment_top_n=3,
    segment_pie_scale=10,
    alpha_by='embedding',                                               
    alpha_range=(0.2, 1.0),                
    alpha_clip=None,                       
    prioritize_high_values=True,
    boundary_linewidth=0.5,
    alpha=1,
    plot_boundaries=True,
)